In [1]:
# Colab cell
try:
    import google.colab
    IN_COLAB = True
    !wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py
    !pip install -q -U zarr
    !pip install -q -U fsspec
    !pip install -q -U gcsfs
    !pip install -q -U xarray
except:
    IN_COLAB = False

In [21]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamaño en GB:", ds.nbytes / 1e9)

Tamaño en GB: 19.324476644


In [3]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pickle

from src.ml_training import *

# from dask_ml.wrappers import ParallelPostFit

def to_pickle(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
def read_pickle(fn):
    with open(fn, 'rb') as f:
        return pickle.load(f)
    
results = 'results'
verbose = False

import os
if not os.path.exists(results):
    os.makedirs(results)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
z_500 = ds.z.sel(level=500).drop('level')
z_500 = z_500.rename('z_500')

z_850 = ds.z.sel(level=850).drop('level')
z_850 = z_850.rename('z_850')

z_1000 = ds.z.sel(level=1000).drop('level')
z_1000 = z_1000.rename('z_1000')

q_500 = ds.q.sel(level=500).drop('level')
q_500 = q_500.rename('q_500')

q_850 = ds.q.sel(level=850).drop('level')
q_850 = q_850.rename('q_850')

q_1000 = ds.q.sel(level=1000).drop('level')
q_1000 = q_1000.rename('q_1000')

pv_500 = ds.pv.sel(level=500).drop('level')
pv_500 = pv_500.rename('pv_500')

pv_850 = ds.pv.sel(level=850).drop('level')
pv_850 = pv_850.rename('pv_850')

pv_1000 = ds.pv.sel(level=1000).drop('level')
pv_1000 = pv_1000.rename('pv_1000')

t_500 = ds.t.sel(level=500).drop('level')
t_500 = t_500.rename('t_500')

t_850 = ds.t.sel(level=850).drop('level')
t_850 = t_850.rename('t_850')

t_1000 = ds.t.sel(level=1000).drop('level')
t_1000 = t_1000.rename('t_1000')

t2m   = ds.t2m

data = xr.merge([z_500, z_850, z_1000, t_500, t_850, t_1000, t2m, q_500, q_850, q_1000, pv_500, pv_850, pv_1000])

data_train = data.sel(time=slice('1979', '2016'))
data_test  = data.sel(time=slice('2017', '2018'))

z_test   = load_test_data('z', ds=ds)
t_test   = load_test_data('t', ds=ds)
t_test = t_test.rename('t_850')
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

time, nlat, nlon = data_train.z_500.shape

time, nlat, nlon

(333120, 12, 14)

In [22]:

z_test   = load_test_data('z', ds=ds)
t_test   = load_test_data('t', ds=ds)
t_test = t_test.rename('t_850')
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

Selecting from ds...
Selecting from ds...


In [14]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std  = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test  = (data_test - data_mean)  / data_std

In [15]:
# Inputs and Outputs
experiments = [
    [['t_850'],              ['t_850']],
#     [['t_500', 't_850', 't_1000'],             ['t_850']],
#     [['z_500', 't_850'],        ['t_850']],
#     [['z_850', 't_850'], ['t_850']],
    [['z_500', 'z_850', 'z_1000', 't_500', 't_850', 't_1000'], ['t_850']],
    [['z_500', 'z_850', 'z_1000', 't_500', 't_850', 't_1000', 'q_500', 'q_850',
      'q_1000', 'pv_500', 'pv_850', 'pv_1000'], ['t_850']],
#     [['t2m'],           ['t2m']],
#     [['z', 't'],        ['t2m']],
#     [['z', 't', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time
lead_time_str = '3d'

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV
from sklearn.linear_model import MultiTaskElasticNet, MultiTaskElasticNet

preds  = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

models = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

train_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

test_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

weighted_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

model_args = [
    [LinearRegression,    'lr', {}], 
#     [Ridge,               'ridge', {}], 
#     [RidgeCV,             'ridge_cv', {'alphas': [0.001, 0.01, 1., 10., 50.]}],
#     [MultiTaskLasso,      'lasso', {'tol' : 0.001, 'selection' : 'random'}],
    [MultiTaskElasticNet, 'elasticnet', {'tol' : 0.001, 'selection' : 'random'}]
]

In [23]:
for model in model_args:
    model_fun, model_str, extra_args = model
    
    print('Running:', model_str)
    for idx, (i, o) in enumerate(experiments):
        print(f'{idx}: Input variables = {i}; output variables = {o}')
        
        X_train, y_train, X_test, y_test, time = create_X_y_time(input_vars=i, 
                                                                 output_vars=o, 
                                                                 data=[data_train, data_test], 
                                                                 lead_time=lead_time, 
                                                                 data_subsample=5)
        
        data_needed_train = [X_train, y_train, X_test, y_test, nlat, nlon]
        
        predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                       data=data_needed_train,
                                                                       num_outputs=len(o),
                                                                       extra_args=extra_args,
                                                                       verbose=False)
        
        predictions = unnormalize_preds(preds=predictions, 
                                        output_vars=o, 
                                        valid_time=time, 
                                        lat_lon=[data_train.lat, data_train.lon],
                                        data_std=data_std, 
                                        data_mean=data_mean)

        preds[model_str].append(predictions)
        models[model_str].append(model_res)

        # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
        r = compute_weighted_rmse(predictions, test_data).compute()
        print(r)
        if verbose:
            print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')
        
        train_mse[model_str].append(mse_train)
        test_mse[model_str].append(mse_test)
        weighted_mse[model_str].append(r)

        # Save models
        preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
        model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
        
        predictions.to_netcdf(preds_nc_str)
        to_pickle(model_res, model_res_str)
    print('----------------------------------------------------------------------------')

Running: lr
0: Input variables = ['t_850']; output variables = ['t_850']
<xarray.Dataset>
Dimensions:  ()
Data variables:
    t_850    float64 3.3
1: Input variables = ['z_500', 'z_850', 'z_1000', 't_500', 't_850', 't_1000']; output variables = ['t_850']
<xarray.Dataset>
Dimensions:  ()
Data variables:
    t_850    float64 3.188
2: Input variables = ['z_500', 'z_850', 'z_1000', 't_500', 't_850', 't_1000', 'q_500', 'q_850', 'q_1000', 'pv_500', 'pv_850', 'pv_1000']; output variables = ['t_850']
<xarray.Dataset>
Dimensions:  ()
Data variables:
    t_850    float64 3.22
----------------------------------------------------------------------------
Running: elasticnet
0: Input variables = ['t_850']; output variables = ['t_850']
<xarray.Dataset>
Dimensions:  ()
Data variables:
    t_850    float64 3.823
1: Input variables = ['z_500', 'z_850', 'z_1000', 't_500', 't_850', 't_1000']; output variables = ['t_850']
<xarray.Dataset>
Dimensions:  ()
Data variables:
    t_850    float64 3.667
2: Input 

In [24]:
import pandas as pd

results_df = pd.DataFrame(columns=['model', 'input', 'output',  'train_mse', 'test_mse', 'weighted_mse'])

block_rows = []
for key in models.keys():
    # repeat key as many time as needed to have a long df
    first_three_cols = np.hstack((np.array([key] * len(experiments)).reshape(len(experiments),1), 
                                  np.array(experiments, dtype='object')))
    block_rows.append(first_three_cols)
    
block_rows = np.array(block_rows).reshape(len(models.keys()) * len(experiments), 3) 

results_df['model'] = block_rows[:, 0]
results_df['input'] = block_rows[:, 1]
results_df['output'] = block_rows[:, 2]


train_mse_col = []
for model, ds_list in train_mse.items():
    train_mse_col += ds_list

test_mse_col = []
for model, ds_list in test_mse.items():
    test_mse_col += ds_list
    

weighted_mse_col = []
for model, ds_list in weighted_mse.items():
    model_metrics = []
    for ds in ds_list:
        exp_metrics = []
        for var in list(ds.variables):
            exp_metrics.append(round(float(ds[var].values), 4))

        model_metrics.append(exp_metrics)
    weighted_mse_col += model_metrics

    
print('\nNumber of rows equals number of metrics? ', results_df.shape[0] == len(train_mse_col))

results_df['train_mse'] = train_mse_col
results_df['test_mse'] = test_mse_col

results_df['weighted_mse'] = weighted_mse_col
# results_df['weighted_mse'] = results_df['weighted_mse'].apply(lambda x : x[0])
results_df


Number of rows equals number of metrics?  False


ValueError: Length of values (12) does not match length of index (6)

In [80]:
weighted_mse_col

[[], [], [], [], []]